# Example: VQE program

This tutorial will be demonstation of creating VQE Program as well as migration guide on how you can replicate IBM Quantum VQE custom runtime program.

Let's first get information on what is VQE runtime program and what inputs and outputs it has.

**Description** of runtime program is: Variational Quantum Eigensolver (VQE) to find the minimal eigenvalue of a Hamiltonian.

**Inputs**:

| name | type | description |
| ---- | ---- | ----------- |
| ansatz | object | A parameterized quantum circuit preparing the ansatz wavefunction for the VQE. It is assumed that all qubits are initially in the 0 state. | 
| initial_parameters|[array,string]|Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters.|
|operator|object|The Hamiltonian whose smallest eigenvalue we're trying to find. Should be PauliSumOp|
|method|str|The classical optimizer used in to update the parameters in each iteration. |


**Return values**

| name | type | description |
| ---- | ---- | ----------- |
|cost_function_evals|integer|The number of cost function (energy) evaluations.|
|optimal_parameters|null|Not supported at the moment, therefore ``None``.|
|optimal_point|array|The optimal parameter values found during the optimization. This is a numpy array.|
|optimal_value|number|The smallest value found during the optimization. Equal to the ``eigenvalue`` attribute. This is a float.|
|optimizer_evals|integer|The number of steps of the optimizer.|
|optimizer_history|object|A dictionary containing information about the function evaluations (not necessarily the actual parameter value!): the current evaluation count, the parameters, the energy and the standard deviation.|
|optimizer_time|number|The total time taken by the optimizer. This is a float.|

We will also add optional `QiskitRuntimeService` as an argument to use that to access real devices.


With that information we can start drafting our program implementation in `vqe.py` file.

What our program should do:

1. parse input arguments
2. create run_vqe function that accepts estimator instance, creates VQE and runs calculation
3. decide which estimator to use and run vqe
    - if runtime service was passed then create a session and run `run_vqe` function
    - if runtime service was not passed then use stantard qiskit estimator
4. save results from vqe

Roughly our VQE program will look like this. Full code can be found in [vqe.py](./source_files/vqe/vqe.py) file.

```python
# vqe.py

import ...

def run_vqe(
    initial_parameters,
    ansatz,
    operator,
    estimator,
    method
):
    ...

arguments = get_arguments()

service = arguments.get("service")
ansatz = arguments.get("ansatz")
operator = arguments.get("operator")
initial_parameters = arguments.get("initial_parameters") 
optimizer = ...

...

if service is not None:
    # if we have service we need to open a session and create estimator
    backend = arguments.get("backend", "ibmq_qasm_simulator")
    with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, options=options) # qiskit_ibm_runtime.Estimator
        vqe_result = run_vqe( estimator=estimator, ...)
else:
    # if we do not have a service let's use standart local estimator
    estimator = QiskitEstimator() # qiskit.primitives.Estimator

vqe_result, callback_dict = run_vqe(
    initial_parameters=initial_parameters,
    ansatz=ansatz,
    operator=operator,
    estimator=estimator,
    method=method
)

save_result({
    "optimal_point": vqe_result.x.tolist(),
    "optimal_value": vqe_result.fun,
    "optimizer_evals": vqe_result.nfev,
    "optimizer_history": callback_dict.get("cost_history", []),
    "optimizer_time": callback_dict.get("_total_time", 0)
})


```

At this point we have our program implemented. Now we need to actually run it. But before let's prepare input arguments from our VQE program.

In [1]:
import numpy as np


from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

USE_RUNTIME_SERVICE = False

service = None
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService()
    backend = "ibmq_qasm_simulator"

operator = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
)
ansatz = EfficientSU2(operator.num_qubits)

input_arguments = {
    "ansatz": ansatz,
    "operator": operator,
    "method": "COBYLA",
    "service": service,
}

input_arguments

{'ansatz': <qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 at 0x7ff1d0537f40>,
 'operator': SparsePauliOp(['YZ', 'ZI', 'ZZ', 'XX'],
               coeffs=[ 0.398 +0.j, -0.398 +0.j, -0.0113+0.j,  0.181 +0.j]),
 'method': 'COBYLA',
 'service': None}

With arguments prepared we can create our quantum serverless client, setup provider and run our program

In [2]:
from quantum_serverless import ServerlessProvider
import os

In [3]:
serverless = ServerlessProvider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    # token=os.environ.get("GATEWAY_TOKEN", "<TOKEN>"), # token can be used instead of user/password combination
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)
serverless

<ServerlessProvider: gateway-provider>

In [6]:
from quantum_serverless import Program

program = Program(title="vqe", entrypoint="vqe.py", working_dir="./source_files/vqe/")

serverless.upload(program)

'vqe'

In [7]:
job = serverless.run("vqe", arguments=input_arguments)
job

<Job | 06c821c3-5f86-48f8-87b5-98e21a553d08>

In [8]:
job.status()

'QUEUED'

In [9]:
job.result()

{'result': {'0': 0.4266, '1': 0.3574, '2': 0.104, '3': 0.112},
 'optimal_point': [2.400955119274195,
  2.717799958252893,
  1.3628585897978087,
  2.4311917260940845,
  5.754283033860017,
  1.1224006163027451,
  6.624675068668877,
  4.76802186154173,
  0.7666935155193715,
  3.627167675726581,
  1.6068996965600872,
  -0.07959435779430134,
  2.790199999683474,
  4.248957104589368,
  2.961996372880535,
  5.240096622392362],
 'optimal_value': -0.702930389778663,
 'optimizer_time': 5.314694999993662}